In [0]:
import pandas as pd
import sklearn as sk
import numpy as np

Input data:
- blackbox features
- 300 dimensions
- input data size: 200
- to predict: 19.750

In [0]:
df = pd.read_csv('train.csv')
print("Shape: ", df.shape)
train_labels = df['target'].values
train_data = df.drop(['id', 'target'], axis=1).values
df.head()

Shape:  (250, 302)


,id,target,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0,1.0,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,-2.246,1.825,-0.912,-0.107,0.305,0.102,0.826,0.417,0.177,-0.673,-0.503,1.864,0.410,-1.927,0.102,-0.931,1.763,1.449,-1.097,-0.686,-0.250,-1.859,1.125,1.009,-2.296,0.385,-0.876,1.528,-0.144,-1.078,...,-0.681,1.250,-0.565,-1.318,-0.923,0.075,-0.704,2.457,0.771,-0.460,0.569,-1.320,-1.516,-2.145,-1.120,0.156,0.820,-1.049,-1.125,0.484,0.617,1.253,1.248,0.504,-0.802,-0.896,-1.793,-0.284,-0.601,0.569,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,1,0.0,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,0.004,-0.291,2.907,1.085,2.144,1.540,0.584,1.133,1.098,-0.237,-0.498,0.283,-1.100,-0.417,1.382,-0.515,-1.519,0.619,-0.128,0.866,-0.540,1.238,-0.227,0.269,-0.390,-2.721,1.659,0.106,-0.121,1.719,...,0.971,-1.489,0.530,0.917,-0.094,-1.407,0.887,-0.104,-0.583,1.267,-1.667,-2.771,-0.516,1.312,0.491,0.932,2.064,0.422,1.215,2.012,0.043,-0.307,-0.059,1.121,1.333,0.211,1.753,0.053,1.274,-0.612,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,2,1.0,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,0.137,0.183,0.459,0.478,-0.425,0.352,1.095,0.300,-1.044,0.270,-1.038,0.144,-1.658,-0.946,0.633,-0.772,1.786,0.136,-0.103,-1.223,2.273,0.055,-2.032,-0.452,0.064,0.924,-0.692,-0.067,-0.917,1.896,...,-0.540,-0.299,1.074,-0.748,1.086,-0.766,-0.931,0.432,1.345,-0.491,-1.602,-0.727,0.346,0.780,-0.527,-1.122,-0.208,-0.730,-0.302,2.535,-1.045,0.037,0.020,1.373,0.456,-0.277,1.381,1.843,0.749,0.202,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,3,1.0,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,0.503,0.274,0.335,-1.148,0.067,-1.010,1.048,-1.442,0.210,0.836,-0.326,0.716,-0.764,0.248,-1.308,2.127,0.365,0.296,-0.808,1.854,0.118,0.380,0.999,-1.171,2.798,0.394,-1.048,1.078,0.401,-0.486,...,-0.083,-0.831,1.251,-0.206,-0.933,-1.215,0.281,0.512,-0.424,0.769,0.223,-0.710,2.725,0.176,0.845,-1.226,1.527,-1.701,0.597,0.150,1.864,0.322,-0.214,1.282,0.408,-0.910,1.020,-0.299,-1.574,-1.618,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,4,1.0,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,-0.012,2.198,0.190,0.453,0.494,1.478,-1.412,0.270,-1.312,-0.322,-0.688,-0.198,-0.285,1.042,-0.315,-0.478,0.024,-0.190,1.656,-0.469,-1.437,-0.581,-0.308,-0.837,-1.739,0.037,0.336,-1.102,2.371,0.554,...,-1.050,-0.347,0.904,-1.324,-0.849,3.432,0.222,0.416,0.174,-1.517,-0.337,0.055,-0.464,0.014,-1.073,0.325,-0.523,-0.692,0.190,-0.883,-1.830,1.408,2.319,1.704,-0.723,1.014,0.064,0.096,-0.775,1.845,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


StandardScaler will transform the data such that its distribution will have a mean value 0 and standard deviation of 1.

In [0]:
from sklearn.preprocessing import StandardScaler
train_data = StandardScaler().fit_transform(train_data)
train_data

array([[-0.12173567,  2.17600225,  0.5036917 , ..., -0.42172402,
         1.02241106, -0.96572011],
       [ 1.06157697, -0.93927777, -0.53978988, ..., -1.1087964 ,
        -0.93965825,  1.13688315],
       [-0.54829027, -0.06167818, -0.50546483, ..., -0.00245942,
         0.78584951, -1.11634821],
       ...,
       [ 1.39278408, -1.42771556, -0.04747057, ...,  0.75582776,
         0.15766927, -0.60875213],
       [ 0.59889069,  1.05914755,  0.01627596, ..., -0.81390694,
         2.00742153, -0.30336913],
       [ 0.46741151,  0.42675961, -0.02785625, ..., -1.46487041,
        -0.63053791,  1.65376453]])

<strong>Recursive feature elimination</strong> is a backward selection of the predictors. This technique begins by building a model on the entire set of predictors and computing an importance score for each predictor. The least important predictor are then removed, the model is re-built, and importance scores are computed again. 



In [0]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFE
from sklearn.feature_selection import f_classif
from sklearn.svm import LinearSVC

n_features = 25
selector_name = "rfe"
selectors = {
    "k_best": SelectKBest(f_classif, k=n_features),
    "rfe": RFE(LinearSVC(max_iter=5000), n_features)
}
selector = selectors[selector_name]

selected_features = selector.fit_transform(train_data, train_labels)

if hasattr(selector, 'scores_'):
  feature_indexes = (-selector.scores_).argsort()[:n_features]
else:
  feature_indexes = selector.ranking_.argsort()[:n_features]

print(feature_indexes)
selected_train_data = train_data[:, feature_indexes]

[227 264 272  91  33 261  95 258  98 281 199  73 194 101 285 108  43 298
  46 209  65 295 217  13 228]


In [0]:
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=5)
n_estimators = 10

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

log_clf = LogisticRegression()
param_grid = {
  'class_weight' : ['balanced', None], 
  'penalty' : ['l2','l1'], 
  'solver': ['liblinear'] ,
  'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

grid = GridSearchCV(estimator = log_clf, cv=3, scoring="roc_auc", param_grid = param_grid, n_jobs = -1)

grid.fit(selected_train_data, train_labels)

print("Best Score:" + str(grid.best_score_))
print("Best Parameters: " + str(grid.best_params_))

best_parameters = grid.best_params_

model = grid.best_estimator_

Best Score:0.876338784067086
Best Parameters: {'C': 1, 'class_weight': None, 'penalty': 'l2', 'solver': 'liblinear'}


Bagging: Due to the theoretical variance of the training dataset (we remind that a dataset is an observed sample coming from a true unknown underlying distribution), the fitted model is also subject to variability: if another dataset had been observed, we would have obtained a different model.
<strong>The idea of bagging is then simple: we want to fit several independent models and “average” their predictions in order to obtain a model with a lower variance.</strong>

Boosting: Intuitively, <strong>each new model focus its efforts on the most difficult observations</strong> to fit up to now, so that we obtain, at the end of the process, a strong learner with lower bias (even if we can notice that boosting can also have the effect of reducing variance). 

In [0]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

base_estimator = DecisionTreeClassifier()

model = BaggingClassifier(base_estimator=base_estimator, n_estimators=n_estimators)

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=n_estimators)

In [0]:
results = model_selection.cross_val_score(model, selected_train_data, train_labels, cv=kfold)
print(results)

In [0]:
model.fit(selected_train_data, train_labels)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=10,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
df = pd.read_csv('test.csv')
ids = df['id'].values
test_data = df.drop('id', axis = 1).values
selected_test_data = StandardScaler().fit_transform(test_data[:, feature_indexes])

In [0]:
predicted = model.predict_proba(selected_test_data)
predicted = [p[1] for p in predicted]
submission_df = pd.DataFrame({'id': ids, 'target': predicted})
submission_df.head()
submission_df.to_csv('submission.csv', index=False)

<h2>Average Submissions</h2>


In [0]:
import os
submissions_dir = "./submissions"
files = os.listdir(submissions_dir)

sub_df = [pd.read_csv(submissions_dir + "/" + file) for file in files if '.csv' in file]
mean_df = pd.concat(sub_df).groupby(level=0).mean()
mean_df
mean_df.to_csv('submission.csv', index=False)